# Python + Fortran90

Luego de realizar varias optimizaciones en los algoritmos utilizados en las simulaciones, decidimos migrar algunas pocas funciones a Fortran90. 

En este notebook se muestran los distintos profiles obtenidos con estas nuevas funciones. En los profiles solo se van a mostrar 20 funciones ordenas en orden decresiente de tiempo computacional.

In [1]:
import pstats

## Primera versión del código

Escrita totalmente en python.

In [2]:
p = pstats.Stats('data/prop')
p.sort_stats('tottime').print_stats(20)

Mon Jun 13 16:15:04 2016    data/prop

         21748402 function calls (21746006 primitive calls) in 351.969 seconds

   Ordered by: internal time
   List reduced from 881 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  4191930  124.165    0.000  166.385    0.000 /home/fnbellomo/Desktop/ten/ten/mechanisms.py:162(__transfer_rate)
  2246171   90.432    0.000  112.874    0.000 /home/fnbellomo/Desktop/ten/ten/utils.py:15(generate_random_points_in_sphere)
  6371073   53.849    0.000   53.849    0.000 {built-in method builtins.sum}
  2062465   31.399    0.000  152.377    0.000 /home/fnbellomo/Desktop/ten/ten/exciter.py:124(walk)
    33500   27.871    0.001  347.449    0.010 /home/fnbellomo/Desktop/ten/ten/mechanisms.py:27(forster)
  4375636   14.586    0.000   14.586    0.000 {method 'random_sample' of 'mtrand.RandomState' objects}
  2246171    8.672    0.000    8.672    0.000 {method 'randn' of 'mtrand.RandomState' objects}
       28

Podemos observar que la función `__transfer_rate` (que calcula la taza de transferencia del exiton a cualquier aceptor mediante un mecanismo de forster) tarda 124 seg. Es la función que más tiempo acumula durante toda la corrida.
Notar que esta verción original **tarda 352 seg**.

## Transfer en Fortran

Ahora, la misma simulación, pero con esta función implementada en Fortran90

In [3]:
p = pstats.Stats('data/prop2')
p.sort_stats('tottime').print_stats(20)

Tue Jun 14 09:37:11 2016    data/prop2

         13779389 function calls (13776987 primitive calls) in 203.436 seconds

   Ordered by: internal time
   List reduced from 896 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  2315887   93.238    0.000  115.835    0.000 /home/fnbellomo/Desktop/ten/ten/utils.py:15(generate_random_points_in_sphere)
    34000   39.293    0.001  198.823    0.006 /home/fnbellomo/Desktop/ten/ten/mechanisms/mechanisms.py:29(forster)
  2128783   32.715    0.000  158.611    0.000 /home/fnbellomo/Desktop/ten/ten/exciter.py:124(walk)
  4512670   14.714    0.000   14.714    0.000 {method 'random_sample' of 'mtrand.RandomState' objects}
  2247859   13.619    0.000   13.619    0.000 {built-in method builtins.sum}
  2315887    8.803    0.000    8.803    0.000 {method 'randn' of 'mtrand.RandomState' objects}
       28    0.193    0.007  203.163    7.256 /home/fnbellomo/Desktop/ten/ten/experiments.py:13(quenching)
   

Esta nueva función no alcanza a rankear entre las 20 funciones más costosas. Ademas, el método `{built-in method builtins.sum}` tambien redujo dastricamente su tiempo (de 53 seg a 13 seg).
Si comparamos los tiempos totales, en el primer caso es de 352 segundos mientras que en el segundo caso es de 203 segundos. 

Esta nueva implementación demora **57% de la versión original**.

## Generando puntos aleatorios desde Fortran

Ahora migramos la función encargada de generar puntos aleatorios en una esfera. Esta función se utiliza para crear la posición inicial de los aceptores, del exiton y también se utiliza para realizar el random walk.

In [4]:
p = pstats.Stats('data/prop3')
p.sort_stats('tottime').print_stats(20)

Wed Jun 15 08:57:55 2016    data/prop3

         10237062 function calls (10234654 primitive calls) in 93.374 seconds

   Ordered by: internal time
   List reduced from 896 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  2333560   34.981    0.000   58.027    0.000 /home/fnbellomo/Desktop/ten/ten/exciter.py:124(walk)
    35500   33.268    0.001   92.262    0.003 /home/fnbellomo/Desktop/ten/ten/mechanisms/mechanisms.py:29(forster)
  2576240   13.651    0.000   13.651    0.000 {built-in method builtins.sum}
  2647268    9.690    0.000    9.724    0.000 /home/fnbellomo/Desktop/ten/ten/utils.py:17(generate_random_points_in_sphere)
  2404560    0.967    0.000    0.967    0.000 {method 'random_sample' of 'mtrand.RandomState' objects}
       28    0.136    0.005   93.129    3.326 /home/fnbellomo/Desktop/ten/ten/experiments.py:13(quenching)
    35500    0.106    0.000    0.388    0.000 /home/fnbellomo/Desktop/ten/ten/nanoparticle.py:207(e

Si bien la función `walk` continua estando entre las que más tarda, ahora tarda un 10% de la función original. Ademas, los métodos `{method 'random_sample' of 'mtrand.RandomState' objects}` y `{method 'randn' of 'mtrand.RandomState' objects}` tambien redujeron sus tiempos de ejecución.

Esta segunda implementación demora **26% de la versión original**.

## La caminata

Migrando el random walk a Fortran obtenemos

In [5]:
p = pstats.Stats('data/prop4')
p.sort_stats('tottime').print_stats(20)

Wed Jun 15 09:22:16 2016    data/prop4

         4751282 function calls (4748875 primitive calls) in 36.753 seconds

   Ordered by: internal time
   List reduced from 880 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    33500   28.142    0.001   35.702    0.001 /home/fnbellomo/Desktop/ten/ten/mechanisms/mechanisms.py:29(forster)
  2176426    6.793    0.000    6.793    0.000 /home/fnbellomo/Desktop/ten/ten/exciter.py:125(walk)
  2243426    0.767    0.000    0.767    0.000 {method 'random_sample' of 'mtrand.RandomState' objects}
    67028    0.272    0.000    0.309    0.000 /home/fnbellomo/Desktop/ten/ten/utils.py:17(generate_random_points_in_sphere)
       28    0.121    0.004   36.509    1.304 /home/fnbellomo/Desktop/ten/ten/experiments.py:13(quenching)
    33500    0.092    0.000    0.275    0.000 /home/fnbellomo/Desktop/ten/ten/exciter.py:14(__init__)
    33500    0.089    0.000    0.364    0.000 /home/fnbellomo/Desktop/ten/te

De donde **walk** pasa de tardar 34 seg a 6 seg. Ademas, varias métodos internos desaparecen.

Con la migración de esta terceer función, la simulación demora **10% de la versión original**.

##  Mecanismo de transferencia

Ahora, la implementación total del mecanismo de trasferencia es en Fortran

In [6]:
p = pstats.Stats('data/prop5')
p.sort_stats('tottime').print_stats(20)

Wed Jun 15 17:22:12 2016    data/prop5

         330851 function calls (328444 primitive calls) in 8.106 seconds

   Ordered by: internal time
   List reduced from 878 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    33500    7.141    0.000    7.141    0.000 ../../ten/mechanisms/mechanisms.py:29(forsterF90)
    67028    0.250    0.000    0.281    0.000 ../../ten/utils.py:17(generate_random_points_in_sphere)
       28    0.108    0.004    7.868    0.281 ../../ten/experiments.py:13(quenching)
    33500    0.084    0.000    0.329    0.000 ../../ten/nanoparticle.py:207(excite)
    33500    0.079    0.000    0.245    0.000 ../../ten/exciter.py:12(__init__)
    33500    0.065    0.000    0.165    0.000 ../../ten/exciter.py:80(laser_generated)
    33528    0.064    0.000    0.245    0.000 ../../ten/aceptor.py:66(generate)
    33500    0.046    0.000    0.290    0.000 ../../ten/nanoparticle.py:162(doped)
      121    0.034    0.000    0

El tiempo de `walk` y `forster` están sumados en una sola función (`forsterF90`) que es la que más tiempo insume. Entonces, la mayor parte del tiempo de esta implementación es debido a `walk`.

Esta última implementación demora **2% de la versión original**.

In [7]:
#Este css es trabajo de @LorenaABarba y su grupo
from IPython.core.display import HTML
css_file = '../css/personal.css'
HTML(open(css_file, "r").read())

##### Licencia
El código esta licenciado bajo MIT.

La documentación bajo:

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">TEN</span> by <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">Laboratorio de Microscopia Óptica Avanzada - UNRC</span> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/pewen/ten" rel="dct:source">https://github.com/pewen/ten</a>